# 验证 Inference 模型预测的 Local Mask

本notebook用于加载保存的inference模型，创建环境，并验证模型预测的local mask是否正确。

In [1]:
import os
import sys
import numpy as np
import torch
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 设置显示格式
np.set_printoptions(precision=3, suppress=True)
torch.set_printoptions(precision=3, sci_mode=False)

# 确保当前目录在path中
if not os.getcwd() in sys.path:
    sys.path.append(os.getcwd())

In [2]:
# 导入必要的模块
from fcdl.env.chemical_env import Chemical
from fcdl.model.encoder import make_encoder
from fcdl.model.inference_ours_masking import InferenceOursMask
from fcdl.model.inference_dwm import InferenceDWM
from fcdl.model.inference_ncd import InferenceNCD
from fcdl.utils.utils import TrainingParams, get_env, update_obs_act_spec
from fcdl.utils.replay_buffer import ReplayBuffer

## 1. 加载保存的参数和模型

In [3]:
# 设置模型路径
name = "ncd-1"
model_path = f"data1/iwhwang/causal_rl/Chemical/{name}/trained_models/inference_15k"
params_path = f"data1/iwhwang/causal_rl/Chemical/{name}/params"
env_params_path = f"data1/iwhwang/causal_rl/Chemical/{name}/params"

# 检查文件是否存在
assert os.path.exists(model_path), f"模型文件不存在: {model_path}"
assert os.path.exists(params_path), f"参数文件不存在: {params_path}"
assert os.path.exists(env_params_path), f"环境参数文件不存在: {env_params_path}"

In [4]:
# 加载参数
params_dict = torch.load(params_path)
params = TrainingParams(training_params_fname="policy_params.json", train=False)

# 将加载的参数字典复制到params对象
for key, value in params_dict.items():
    setattr(params, key, value)

# 加载环境参数
env_params = torch.load(env_params_path)
print(f"已加载参数和环境设置")

已加载参数和环境设置


In [9]:
classical_params = TrainingParams(training_params_fname="policy_params.json", train=False)

In [20]:
# Find the difference between classical_params and params
def get_param_dict(obj):
    return {k: v for k, v in obj.__dict__.items() if not k.startswith('_')}

classical_dict = get_param_dict(classical_params)
params_dict = get_param_dict(params)

# Create a function to safely compare objects
def compare_values(v1, v2):
    """Compare values, handling unhashable types"""
    try:
        return v1 == v2
    except:
        return str(v1) == str(v2)

# Store differences in a list (not a set)
differences = []

# Find keys that are in params but not in classical_params or have different values
for key, value in params_dict.items():
    if key not in classical_dict:
        differences.append((key, "Only in params", str(value)))
    elif not compare_values(classical_dict[key], value):
        differences.append((key, "Different values", 
                          f"params: {str(value)}", 
                          f"classical: {str(classical_dict[key])}"))

# Find keys that are in classical_params but not in params
for key, value in classical_dict.items():
    if key not in params_dict:
        differences.append((key, "Only in classical_params", str(value)))

# Display the differences
for diff in differences:
    print(diff)

('seed', 'Different values', 'params: 1', 'classical: -1')
('ours_params', 'Different values', "params: {'feature_fc_dims': [128, 128], 'generative_fc_dims': [128, 128], 'vq_encode_fc_dims': [128, 64], 'vq_decode_fc_dims': [32], 'ncd_fc_dims': [128, 128], 'code_labeling': True, 'vqvae_ema': True, 'ema': 0.99, 'codebook_size': 16, 'code_dim': 16, 'reg_coef': 0.001, 'vq_coef': 1.0, 'commit_coef': 0.25, 'local_mask_sampling_num': 1, 'eval_local_mask_sampling_num': 1}", "classical: {'feature_fc_dims': [128, 128], 'generative_fc_dims': [128, 128], 'vq_encode_fc_dims': [128, 64], 'vq_decode_fc_dims': [32], 'ncd_fc_dims': [128, 128], 'code_labeling': False, 'vqvae_ema': True, 'ema': 0.99, 'codebook_size': 16, 'code_dim': 16, 'reg_coef': 0.001, 'vq_coef': 1.0, 'commit_coef': 0.25, 'local_mask_sampling_num': 1, 'eval_local_mask_sampling_num': 1}")
('training_params', 'Different values', "params: {'inference_algo': 'ncd', 'rl_algo': 'model_based', 'load_id': None, 'load_inference': None, 'load_m

In [18]:
diff

{('action_dim', 50),
 ('action_spec', None),
 ('continuous_action', False),
 ('continuous_factor', False),
 ('continuous_state', False),
 ('device', device(type='cuda')),
 ('feature_inner_dim', (5, 5, 5, 5, 5, 5, 5, 5, 5, 5)),
 ('mute_wandb', False),
 ('num_action_variable', 1),
 ('obs_dims',
  frozenset({('obj0', (5,)),
             ('obj1', (5,)),
             ('obj2', (5,)),
             ('obj3', (5,)),
             ('obj4', (5,)),
             ('obj5', (5,)),
             ('obj6', (5,)),
             ('obj7', (5,)),
             ('obj8', (5,)),
             ('obj9', (5,)),
             ('target_obj0', (5,)),
             ('target_obj1', (5,)),
             ('target_obj2', (5,)),
             ('target_obj3', (5,)),
             ('target_obj4', (5,)),
             ('target_obj5', (5,)),
             ('target_obj6', (5,)),
             ('target_obj7', (5,)),
             ('target_obj8', (5,)),
             ('target_obj9', (5,))})),
 ('obs_spec',
  frozenset({('obj0', (0.0,)),
        

In [5]:
# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
params.device = device
print(f"使用设备: {device}")

使用设备: cuda


## 2. 创建环境和推理模型

In [6]:
# 创建环境
# 强制使用单个环境，不使用向量化环境
params.env_params.num_env = 1  # 确保只使用一个环境
env = get_env(params)
print(f"环境创建完成: {params.env_params.env_name}")

# 更新观测和动作空间
update_obs_act_spec(env, params)

环境创建完成: Chemical


In [ ]:
# 创建编码器和推理模型
encoder = make_encoder(params)
inference = InferenceNCD(encoder, params)

# 加载保存的模型
inference.load(model_path, device)
inference.eval()
print(f"推理模型加载完成")

2025-03-16 09:04:59.657 | INFO     | fcdl.model.inference_ours_masking:__init__:10 - InferenceOursMask
2025-03-16 09:04:59.659 | INFO     | fcdl.model.inference_ours_base:__init__:17 - InferenceOursBase
2025-03-16 09:04:59.659 | INFO     | fcdl.model.inference_ours_base:init_model:44 - set up local causal model
2025-03-16 09:04:59.664 | INFO     | fcdl.model.gumbel:__init__:153 - Set up EMB
2025-03-16 09:04:59.670 | INFO     | fcdl.model.gumbel:__init__:153 - Set up EMB
2025-03-16 09:05:00.393 | INFO     | fcdl.model.inference_ours_base:load:310 - loading inference model from data1/iwhwang/causal_rl/Chemical/ncd-1/trained_models/inference_15k


inference loaded data1/iwhwang/causal_rl/Chemical/ncd-1/trained_models/inference_15k
推理模型加载完成


## 3. 获取样本数据进行预测

In [46]:
# 创建缓冲区来收集样本
buffer = ReplayBuffer(params)
num_samples = 100
current_samples = 0

# 收集样本 - 使用单个环境
obs = env.reset()
done = False

while current_samples < num_samples:
    # 随机选择动作 - 单环境
    action = np.random.randint(0, 5 * 10, size=1)  # 根据环境动作空间调整
    next_obs, reward, done, info = env.step(action.item())
    
    # 添加到缓冲区 - 单环境
    buffer.add(obs, action, reward, next_obs, done, info, True)
    current_samples += 1
    
    # 如果回合结束，重置环境
    if done:
        obs = env.reset()
    else:
        obs = next_obs

print(f"收集了 {current_samples} 个样本")

收集了 100 个样本


In [47]:
from fcdl.utils.utils import preprocess_obs, postprocess_obs
obs, postprocess_obs(preprocess_obs(obs, params))

({'obj0': array([0]),
  'obj1': array([1]),
  'obj2': array([1]),
  'obj3': array([0]),
  'obj4': array([1]),
  'obj5': array([0]),
  'obj6': array([2]),
  'obj7': array([2]),
  'obj8': array([4]),
  'obj9': array([0]),
  'target_obj0': array([0]),
  'target_obj1': array([1]),
  'target_obj2': array([4]),
  'target_obj3': array([0]),
  'target_obj4': array([3]),
  'target_obj5': array([4]),
  'target_obj6': array([1]),
  'target_obj7': array([2]),
  'target_obj8': array([2]),
  'target_obj9': array([0])},
 {'obj0': array([0.], dtype=float32),
  'obj1': array([1.], dtype=float32),
  'obj2': array([1.], dtype=float32),
  'obj3': array([0.], dtype=float32),
  'obj4': array([1.], dtype=float32),
  'obj5': array([0.], dtype=float32),
  'obj6': array([2.], dtype=float32),
  'obj7': array([2.], dtype=float32),
  'obj8': array([4.], dtype=float32),
  'obj9': array([0.], dtype=float32),
  'target_obj0': array([0.], dtype=float32),
  'target_obj1': array([1.], dtype=float32),
  'target_obj2': ar

In [48]:
# 从缓冲区获取样本
batch_size = 13
obs_batch, actions_batch, next_obses_batch, info_batch = buffer.sample_inference(batch_size, "all")

In [49]:
obs_batch['obj0']

tensor([[0.],
        [2.],
        [1.],
        [2.],
        [0.],
        [1.],
        [3.],
        [2.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.]], device='cuda:0')

In [50]:
info_batch['lcms'].shape

torch.Size([13, 3, 10, 11])

In [51]:
info_batch['lcms'][0][0], actions_batch[0] // 5

(tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]], device='cuda:0'),
 tensor([[7],
         [4],
         [5]], device='cuda:0'))

## 4. 验证模型预测的 Local Mask

In [52]:
# 使用推理模型进行预测
with torch.no_grad():
    # 获取模型预测的local mask
    pred_results = inference.eval_local_mask(obs_batch, actions_batch)
    
    # # 提取预测的local mask
    # if hasattr(inference, 'pred_local_mask'):
    #     pred_local_mask = inference.pred_local_mask
    #     print("获取到模型预测的local mask")
    # else:
    #     # 如果模型没有直接暴露pred_local_mask，尝试从pred_results获取
    #     if 'local_mask' in pred_results:
    #         pred_local_mask = pred_results['local_mask']
    #         print("从pred_results获取了local mask")
    #     else:
    #         print("警告：无法获取local mask预测结果")
    #         pred_local_mask = None
    
    # # 提取真实的local mask（如果有）
    # if 'gt_local_mask' in info_batch:
    #     gt_local_mask = info_batch['gt_local_mask']
    #     print("获取到真实的local mask")
    # else:
    #     print("警告：数据中没有真实的local mask")
    #     gt_local_mask = None

In [53]:
pred_results[0].squeeze().shape, info_batch['lcms'].shape

(torch.Size([3, 13, 10, 11]), torch.Size([13, 3, 10, 11]))

In [57]:
(pred_results[0].squeeze().cpu()[0, 1] - info_batch['lcms'][0, 1].cpu()).abs().sum()

tensor(38.)

In [28]:
torch.sigmoid(pred_results[1].squeeze().cpu()[0, 1]), info_batch['lcms'][0, 1].cpu()

(tensor([[0.500, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
          0.500],
         [0.500, 0.500, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
          0.500],
         [0.500, 0.500, 0.500, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
          0.500],
         [0.500, 0.500, 0.500, 0.500, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
          0.500],
         [0.500, 0.500, 0.500, 0.500, 0.500, 0.001, 0.001, 0.001, 0.001, 0.001,
          0.500],
         [0.500, 0.500, 0.199, 0.500, 0.500, 0.500, 0.001, 0.001, 0.001, 0.001,
          0.500],
         [0.500, 0.500, 0.499, 0.498, 0.500, 0.500, 0.500, 0.001, 0.001, 0.001,
          0.500],
         [0.500, 0.500, 0.484, 0.445, 0.018, 0.001, 0.500, 0.500, 0.001, 0.001,
          0.500],
         [0.500, 0.500, 0.004, 0.500, 0.498, 0.043, 0.006, 0.500, 0.500, 0.001,
          0.500],
         [0.500, 0.498, 0.500, 0.499, 0.500, 0.500, 0.500, 0.016, 0.007, 0.500,
          0.500]]),
 tensor([[1., 0., 

In [87]:
torch.tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1.],
        [0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0.]]).abs().sum()

tensor(19.)

In [72]:
pred_results[0].squeeze().cpu()[0] - info_batch['lcms'][0, 0].cpu()

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1.],
        [0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0.]])

In [ ]:
# 分析和可视化local mask（如果有）
if pred_local_mask is not None:
    print("预测的local mask形状:", pred_local_mask.shape)
    
    # 显示第一个样本的预测mask
    sample_idx = 0
    plt.figure(figsize=(10, 8))
    
    # 获取矩阵的维度
    mask_shape = pred_local_mask[sample_idx].shape
    
    # 创建热力图
    sns.heatmap(pred_local_mask[sample_idx].cpu().numpy(), 
                annot=True, 
                fmt=".2f", 
                cmap="YlGnBu",
                xticklabels=[f"Obj {i}" for i in range(mask_shape[1])],
                yticklabels=[f"Obj {i}" for i in range(mask_shape[0])])
    
    plt.title("预测的Local Mask (样本 #0)")
    plt.tight_layout()
    plt.show()
    
    # 如果有真实的local mask，比较准确性
    if gt_local_mask is not None:
        # 计算预测与真实值的差异
        accuracy = (pred_local_mask.round() == gt_local_mask).float().mean().item()
        print(f"Local mask准确率: {accuracy:.4f}")
        
        # 显示真实的local mask
        plt.figure(figsize=(10, 8))
        sns.heatmap(gt_local_mask[sample_idx].cpu().numpy(), 
                    annot=True, 
                    fmt=".0f", 
                    cmap="YlGnBu",
                    xticklabels=[f"Obj {j}" for j in range(mask_shape[1])],
                    yticklabels=[f"Obj {j}" for j in range(mask_shape[0])])
        
        plt.title("真实的Local Mask (样本 #0)")
        plt.tight_layout()
        plt.show()
else:
    print("没有可用的local mask信息进行可视化")

## 5. 额外的分析：检查预测结果

In [ ]:
# 查看所有的预测结果
print("预测结果包含的键:")
for key in pred_results.keys():
    print(f"- {key}: {type(pred_results[key])}")

# 检查预测的状态转移
if 'pred_next_state' in pred_results:
    pred_next_state = pred_results['pred_next_state']
    true_next_state = next_obses_batch
    
    # 计算预测误差
    prediction_error = ((pred_next_state - true_next_state) ** 2).mean().item()
    print(f"\n平均预测误差 (MSE): {prediction_error:.6f}")
    
    # 显示第一个样本的预测与真实值比较
    sample_idx = 0
    
    print(f"\n样本 #{sample_idx} 的预测值与真实值比较:")
    print(f"预测下一状态:\n{pred_next_state[sample_idx]}")
    print(f"真实下一状态:\n{true_next_state[sample_idx]}")

## 6. 探索更多样本的local mask预测

In [ ]:
# 可视化多个样本的local mask预测
if pred_local_mask is not None:
    n_samples = min(4, batch_size)  # 显示最多4个样本
    
    fig, axes = plt.subplots(n_samples, 1, figsize=(10, n_samples * 6))
    
    for i in range(n_samples):
        ax = axes[i] if n_samples > 1 else axes
        
        # 创建热力图
        sns.heatmap(pred_local_mask[i].cpu().numpy(), 
                    annot=True, 
                    fmt=".2f", 
                    cmap="YlGnBu",
                    ax=ax,
                    xticklabels=[f"Obj {j}" for j in range(mask_shape[1])],
                    yticklabels=[f"Obj {j}" for j in range(mask_shape[0])])
        
        ax.set_title(f"预测的Local Mask (样本 #{i})")
    
    plt.tight_layout()
    plt.show()

## 7. 保存分析结果

In [ ]:
# 创建结果目录
results_dir = "local_mask_analysis_results"
os.makedirs(results_dir, exist_ok=True)

# 如果有预测的local mask，将其保存为CSV文件
if pred_local_mask is not None:
    for i in range(min(10, batch_size)):  # 保存前10个样本
        # 转换为DataFrame
        df = pd.DataFrame(pred_local_mask[i].cpu().numpy())
        df.columns = [f"Obj {j}" for j in range(df.shape[1])]
        df.index = [f"Obj {j}" for j in range(df.shape[0])]
        
        # 保存为CSV
        df.to_csv(os.path.join(results_dir, f"local_mask_sample_{i}.csv"))
    
    print(f"已保存local mask预测结果到 {results_dir} 目录")

## 8. 总结和解释

### 分析结论

我们通过以下步骤验证了推理模型预测的local mask:

1. 加载了保存的模型权重和参数
2. 重建了与训练时相同的环境
3. 收集了新的样本数据
4. 使用推理模型生成了local mask预测
5. 可视化并分析了预测结果

**Local Mask 解释**:
- Local mask矩阵中的每个元素表示行对象对列对象的影响程度
- 值接近1表示存在强影响关系，值接近0表示几乎没有影响
- 通过观察local mask，我们可以了解对象之间的因果关系结构

这些分析结果可以帮助我们理解模型是如何捕获环境中的因果关系，以及这些关系的准确性程度。